# Исследование объявлений о продаже квартир

На основе данных сервиса Яндекс.Недвижимость нужно научиться определять рыночную стоимость объектов недвижимости.

**Цель исследования** - установить параметры, которые позволят построить автоматизированную систему для отслеживания аномалий и мошеннической деятельности.

**Ход исследования**

Данные об объектах недвижимости получаем из файла `/datasets/real_estate_data.csv`.

Исследование пройдет в 3 этапа:
* Обзор данных
* Предобработка данных
* Исследовательский анализ данных
* Оформление вывода и результатов работы

## Шаг 1. Обзор данных

Для работы с данными используем библиотеку `pandas`. Импортируем ее прочитаем данные из файла `/datasets/real_estate_data.csv`. В качестве разделителя используем `\t`
Для визуализации результатов и построения графиков используем библиотеку `matplotlib`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
try:
    real_estate_data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
except FileNotFoundError:
    real_estate_data = pd.read_csv('../../datasets/real_estate_data.csv', sep='\t')

Для обзора таблицы с данными выведем на экран первые 10 строк

In [ ]:
real_estate_data.head(10)

Получим общую информацию о таблице

In [ ]:
real_estate_data.info()

В таблице 22 столбца, встречаются различные типы данных.

Согласно документации:
* `airports_nearest` — расстояние до ближайшего аэропорта в метрах (м)
* `balcony` — число балконов
* `ceiling_height` — высота потолков (м)
* `cityCenters_nearest` — расстояние до центра города (м)
* `days_exposition` — сколько дней было размещено объявление (от публикации до снятия)
* `first_day_exposition` — дата публикации
* `floor` — этаж
* `floors_total` — всего этажей в доме
* `is_apartment` — апартаменты (булев тип)
* `kitchen_area` — площадь кухни в квадратных метрах (м²)
* `last_price` — цена на момент снятия с публикации
* `living_area` — жилая площадь в квадратных метрах (м²)
* `locality_name` — название населённого пункта
* `open_plan` — свободная планировка (булев тип)
* `parks_around3000` — число парков в радиусе 3 км
* `parks_nearest` — расстояние до ближайшего парка (м)
* `ponds_around3000` — число водоёмов в радиусе 3 км
* `ponds_nearest` — расстояние до ближайшего водоёма (м)
* `rooms` — число комнат
* `studio` — квартира-студия (булев тип)
* `total_area` — площадь квартиры в квадратных метрах (м²)
* `total_images` — число фотографий квартиры в объявлении

Воспользуемся методом `describe()`, чтобы посмотреть распределение данных по столбцам

In [ ]:
real_estate_data.describe()

Из вывода метода `info()` видно, что в таблице имеются пропуски.
Из вывода метода `describe()` видно, что таблице присутствуют аномалии, например - максимальное и минимальное значения высоты потолков 1 и 100 метров соответственно, большое стандартное отклонение в столбце `total_area` и т.д.

**Выводы**

В каждой строке таблицы мы видим информацию об объекте недвижимости.

При обзоре таблицы с данными замечаем следующие проблемы:
* Имеются пропуски в данных
* Большие стандартные отклонения в столбцах
* Выбросы
* Неправильные типы данных

Чтобы двигаться дальше, необходимо устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Шаг 2.1. Определение и изучение пропущенных значений

В столбце `is_apartment` заменим пропущенные значения на False, т.к. логично предположить, что если не указано, что это апартаменты, значит это не апартаменты

In [ ]:
real_estate_data['is_apartment'] = real_estate_data['is_apartment'].fillna(False)

В столбце `balcony` заменим пропущенные значения на 0, т.к. логично предположить, что если число балконов не указано, значит их нет

In [ ]:
real_estate_data['balcony'] = real_estate_data['balcony'].fillna(0)

В столбце `locality_name` заменим пропуски на 'unknown'

In [ ]:
real_estate_data['locality_name'] = real_estate_data['locality_name'].fillna('unknown')

В столбце `floors_total` имеется небольшое количество пропусков, удалим их

In [ ]:
real_estate_data = real_estate_data.loc[~real_estate_data['floors_total'].isna()]

В столбце `rooms` есть нулевые значения

In [ ]:
real_estate_data.pivot_table(index='rooms', values='total_images', aggfunc='count')

Проверим, являются ли квартиры с 0 комнат студиями

In [ ]:
real_estate_data.query('rooms == 0 and studio == True')

135 квартир из 197 являются студиями. Заменим количество комнат в этих случаях на 1

In [ ]:
real_estate_data.loc[(real_estate_data['rooms'] == 0) & (real_estate_data['studio'] == True), 'rooms'] = 1

In [ ]:
real_estate_data = real_estate_data[real_estate_data['rooms'] != 0]

В остальных столбцах оставим пропуски, т.к. для них нет логичной замены

### Шаг 2.2. Приведение данных к нужным типам

Столбец `last_price` можно привести к целому типу для простоты вычислений, т.к. сумма всех значений этого столбца не имеет дробной части

In [ ]:
real_estate_data['last_price'].sum()

Значения столбцов `floors_total` и `balcony` по логике не могут быть дробными, поэтому их тоже приводим к целому типу. Так же приведем к целому все столбцы, имеющие булев тип данных

In [ ]:

real_estate_data = real_estate_data.astype({'last_price': 'int64',
                                            'floors_total': 'int64',
                                            'balcony': 'int64',
                                            'is_apartment': 'int64',
                                            'studio': 'int64',
                                            'open_plan': 'int64'
                                            })

Проверим таблицу

In [ ]:
real_estate_data.info()

Значения нужных типов, нужные пропуски заполнены, можно двигаться дальше

## Шаг 3. Дополним таблицу новыми значениями

Добавим в таблицу цену квадратного метра и сразу приведем значение к целому типу

In [ ]:
real_estate_data['square_meter_price'] = (real_estate_data['last_price'] / real_estate_data['total_area']).astype('int64')

Добавим в таблицу день недели, месяц и год публикации объявления

In [ ]:
real_estate_data['publication_day'] = real_estate_data['first_day_exposition'].dt.weekday
real_estate_data['publication_month'] = real_estate_data['first_day_exposition'].dt.month
real_estate_data['publication_year'] = real_estate_data['first_day_exposition'].dt.year

Разделим квартиры по этажу на категории. Для этого определим функцию:

In [ ]:
def floor_categorizer(raw):
    if raw['floor'] == 1:
        return 'первый'
    elif raw['floor'] == raw['floors_total']:
        return 'последний'
    return 'другой'

и применим ее к датасету, создав новый столбец

In [ ]:
real_estate_data['floor_category'] = real_estate_data.apply(floor_categorizer, axis=1)

Добавим в таблицу соотношение жилой и общей площади

In [ ]:
real_estate_data['living/total'] = real_estate_data['living_area'] / real_estate_data['total_area']

И отношение площади кухни к общей площади

In [ ]:
real_estate_data['kitchen/total'] = real_estate_data['kitchen_area'] / real_estate_data['total_area']

## Шаг 4. Проведем исследовательский анализ данных и выполним инструкции

### Шаг 4.1. Изучим параметры

**Задание**
Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.

Построим гистограмму по столбцу `total_area`

In [ ]:
real_estate_data.hist('total_area', bins=100);

На гистограмме видим большой хвост справа. Построим диграмму размаха, чтобы убедиться, что это выбросы

In [ ]:
plt.ylim(0, 200)
real_estate_data.boxplot(column='total_area');

Видим выбросы от 0 до 15 и больше 110 кв. м.

Построим гистограмму по столбцу `last_price`

In [ ]:
real_estate_data.hist('last_price', bins=200);

Видим такую же проблему, как и в первом случае. Построим диаграмму размаха

In [ ]:
plt.ylim(-50000, 12500000)
real_estate_data.boxplot(column='last_price');

Построим гистограмму по столбцу `rooms`

In [ ]:
real_estate_data['rooms'].value_counts().plot(kind='bar');

Определим выбросы

In [ ]:
plt.ylim(0, 9)
real_estate_data.boxplot(column='rooms');

Построим гистограмму по столбцу `ceiling_height`

In [ ]:
real_estate_data.hist('ceiling_height', bins=5);

In [ ]:
plt.ylim(2, 3.5)
real_estate_data.boxplot(column='ceiling_height');

Напишем функцию, которая на вход будет принимать столбец датафрейма и возвращать кортеж с верхней и нижней границами диаграммы размаха

In [ ]:
def filter_outliers(column):
    q1 = column.quantile(q=0.25)
    q3 = column.quantile(q=0.75)
    iqr = q3 - q1
    bottom_line = q1 - 1.5 * iqr
    if bottom_line < 0:
        bottom_line = 0
    top_line = q3 + 1.5 * iqr
    return (bottom_line, top_line)

Найдем граничные значения в столбцах `total_area`, `last_price`, `rooms`, `ceiling_height`

In [ ]:
rooms = filter_outliers(real_estate_data['rooms'])
total_area = filter_outliers(real_estate_data['total_area'])
last_price = filter_outliers(real_estate_data['last_price'])
ceiling_height = filter_outliers(real_estate_data['ceiling_height'])

Отфильтруем данные датасета по граничным значениям

In [ ]:
data_filtered = real_estate_data.query('(total_area >= @total_area[0] and total_area <= @total_area[1])'
                                       'and (last_price >= @last_price[0] and last_price <= @last_price[1])'
                                       'and (rooms>= @rooms[0] and rooms <= @rooms[1])'
                                       'and ((ceiling_height >= @ceiling_height[0] and ceiling_height <= @ceiling_height[1]) or ceiling_height.isna())')
data_filtered = data_filtered.reset_index(drop=True)
data_filtered.info()

И построим финальные гистограммы

In [ ]:
data_filtered.hist('total_area');

In [ ]:
data_filtered.hist('last_price');

In [ ]:
data_filtered.hist('rooms');

In [ ]:
data_filtered.hist('ceiling_height');

Мы получили выборку, по которой видно наиболее часто встречающиеся значения общей площади, цены, количества комнат и высоты потолков.

### Шаг 4.2. Изучим время продажи квартиры

**Задание**
Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?

Построим гистограмму

In [ ]:
data_filtered.hist('days_exposition', bins=100);

Чаще всего время продажи квартиры составляет меньше 100 дней

Найдем среднее и медиану времени продажи квартиры

In [ ]:
data_filtered['days_exposition'].agg(['mean', 'median'])

В среднем продажа занимает 168 дней, но 50% квартир продаются менее 90 дней

Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
Чтобы ответить на этот вопрос еще раз взглянем на гистограмму, но уменьшим ее диапазон значений:

In [ ]:
data_filtered.hist('days_exposition', bins=100, range=(0,200))

Самый большой пик количества дней продажи находится в районе 40, следовательно, будем считать, что продажа прошла очень быстро, если время продажи меньше 40 дней, так же видим пик в районе 60 дней - это скорее всего автоматическое снятие объявления по истечению срока публикации

Из первой гистограммы видно, что длительность продажи в 800 дней уже редка, а больше 1000 дней почти нулевая. Будем считать, что продажа была необычно долгой при 1000 и более днях продажи.

### Шаг 4.3. Уберем редкие и выбивающиеся значения

**Задание**
Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.

Отфильтруем данные с временем продажи больше 1000 дней

In [ ]:
data_filtered = data_filtered.query('days_exposition <= 1000')

In [ ]:
data_filtered.hist('days_exposition');

Посмотрим на распределение средних цен и количества объявлений по годам

In [ ]:
data_filtered.pivot_table(index='publication_year', values=['days_exposition', 'last_price'], aggfunc={'days_exposition':'mean', 'last_price':['mean', 'count']})

Видим, что с увеличением года снижается среднее время продажи и цена, а объявлений становится все больше

### Шаг 4.4. Найдем факторы, больше всего влияющие на стоимость квартиры

**Задание**
Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.

Проверим зависимость цены от площади. Для этого построим диаграмму рассеяния и посчитаем коэффициент корреляции

In [ ]:
data_filtered.sort_values('total_area').plot(x='last_price', y='total_area', kind='scatter', alpha=0.5);

In [ ]:
data_filtered['total_area'].corr(data_filtered['last_price'])

Видим, что в среднем, с увеличением площади увеличивается и цена

Посмотрим зависимость цены от количества комнат. Для этого сгруппируем данные по количеству комнат и найдем среднюю цену. Построим график

In [ ]:
data_filtered.pivot_table(index='rooms', values='last_price').plot(grid=True)

В районе 5 комнат есть провал, это связано с малым количеством квартир с 5 и 6 комнатами, в результате чего искажается среднее значение

In [ ]:
data_filtered['rooms'].value_counts()

Из графика видно, что цена зависит от количества комнат, что логично, так как с увеличением количества комнат увеличивается и площадь, а следовательно и цена

Посмотрим зависимость цены от расстояния до центра

Построим диаграмму рассеяния

In [ ]:
data_filtered.sort_values('last_price').plot(x='cityCenters_nearest', y='last_price', kind='scatter', alpha=0.2);

In [ ]:
data_filtered['last_price'].corr(data_filtered['cityCenters_nearest'])

Чем меньше расстояние до центра, тем больше разброс значений цены. Есть тенденция к уменьшению цены при отдалении от центра

Исследуем зависимость цены от этажа, на котором находится квартира

In [ ]:
data_filtered.pivot_table(index='floor_category', values='last_price').plot();

Квартиры на первом этаже дешевле

Построим графики зависимости цены от дня, месяца и года продажи

In [ ]:
data_filtered.pivot_table(index='publication_day', values='last_price').plot(grid=True);

К выходным цена снижается, а с понедельника по пятницу заметен рост с пиком в пятницу

In [ ]:
data_filtered.pivot_table(index='publication_month', values='last_price').plot(grid=True);

Видим большой провал в начале лета, связанный с сезоном отпусков и 2 пика весной и осенью

In [ ]:
data_filtered.pivot_table(index='publication_year', values='last_price').plot(grid=True)
data_filtered.pivot_table(index='publication_year', values='last_price', aggfunc='count').plot(grid=True);

Большой спад цены с 2014 по 2016 годы связан с увеличением числа объявлений в этот период

Таким образом, лучшее время для для размещения объявления - в середине недели весной или осенью

### Шаг 4.5. Рейтинг населенных пунктов

**Задание**
Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце `locality_name`.

In [ ]:
locality = (data_filtered.pivot_table(index='locality_name', values='square_meter_price', aggfunc=['count', 'mean'])
            .sort_values(by=('count', 'square_meter_price'), ascending=False))[:10]
locality.columns = ['count', 'mean_price']
locality

Санкт-Петербург на первом месте и по количеству объявлений и по цене квадратного метра

In [ ]:
locality.query('mean_price == mean_price.max()')

In [ ]:
locality.query('mean_price == mean_price.min()')

Самая низкая цена за квадратный метр в Выборге

### Шаг 4.6. Анализ зависимости цены от расстояния до центра в Санкт-Петербурге

**Задание**
Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (`locality_name`). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.

Отфильтруем всю недвижимость, находящуюся в Санкт-Петербурге

In [ ]:
spb = data_filtered.query('locality_name == "Санкт-Петербург"')

Посмотрим пропуски в столбце `cityCenters_nearest`

In [ ]:
spb['cityCenters_nearest'].isna().sum()

Удалим их

In [ ]:
spb = spb[~spb['cityCenters_nearest'].isna()].reset_index(drop=True)

Добавим новый столбец с целым количеством километров до центра

In [ ]:
spb['to_center_km'] = (spb['cityCenters_nearest'] / 1000).round().astype('int64')

Теперь посчитаем среднюю цену для каждого километра

In [ ]:
spb.pivot_table(index='to_center_km', values='last_price').plot(grid=True);

Устойчивое снижение стоимости наблюдается на расстоянии примерно 8-9 км от центра, следовательно, примем эту зону за центр

### Шаг 4.7. Изучим сегмент квартир в центре

**Задание**
Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

Выделим сегмент квартир в центре

In [ ]:
spb_center = spb.query('to_center_km <= 9')

Выделим нужные для исследования столбцы в отдельную таблицу

In [ ]:
spb_center_sample = spb_center[['last_price',
                                'total_area',
                                'ceiling_height',
                                'rooms',
                                'floors_total',
                                'floor',
                                'floor_category',
                                'publication_day',
                                'publication_month',
                                'publication_year',
                                'to_center_km']]

In [ ]:
spb_center_sample.head()

Посмотрим как распределены значения в центре города следующих параметров: площадь, цена, число комнат, высота потолков. Для этого построим гистограммы

Общая площадь

In [ ]:
spb_center_sample['total_area'].hist(bins=30, label='center', alpha=0.5, color='red',density=True)
data_filtered['total_area'].hist(bins=30, label='all', alpha=0.5, color='blue', density=True)
plt.legend();

Наиболее частое значение общей площади в центре города ~43 квадратных метра. Эта гистограмма похожа на гистограмму выборки по всей базе, но значения немного сдвинулись в большую сторону

Цена

In [ ]:
spb_center_sample['last_price'].hist(bins=30, label='center', alpha=0.5, color='red',density=True)
data_filtered['last_price'].hist(bins=30, label='all', alpha=0.5, color='blue', density=True)
plt.legend();

В центре города преобладают более дорогие квартиры, что логично

Количество комнат

In [ ]:
spb_center_sample['rooms'].hist(label='center', alpha=0.5, color='red',density=True)
data_filtered['rooms'].hist(label='all', alpha=0.5, color='blue', density=True)
plt.legend();

В центре больше продают двухкомнатные квартиры, чем однокомнатные

Высота потолков

In [ ]:
spb_center_sample['ceiling_height'].hist(label='center', alpha=0.5, color='red',density=True)
data_filtered['ceiling_height'].hist(label='all', alpha=0.5, color='blue', density=True)
plt.legend();

Зависимость цены от количества комнат

In [ ]:
spb_center_sample.pivot_table(index='rooms', values='last_price').plot(grid=True);

Чем больше комнат, тем выше цена, как и в общей выборке

In [ ]:
data_filtered.pivot_table(index='rooms', values='last_price').plot(grid=True);

Зависимость цены от этажа

In [ ]:
spb_center_sample.pivot_table(index='floor_category', values='last_price').plot(grid=True);

In [ ]:
data_filtered.pivot_table(index='floor_category', values='last_price').plot(grid=True);

В среднем, цена на квартиры в центре выше и на первом этаже квартиры дешевле

Зависимость цены от расстояния до центра

In [ ]:
spb_center_sample['to_center_km'].corr(spb_center_sample['last_price'])

In [ ]:
data_filtered['cityCenters_nearest'].corr(data_filtered['last_price'])

В центре города зависимость цены от расстояния до центра выражена меньше, чем в общей выборке

Зависимость цены от дня, месяца и года публикации объявления

In [ ]:
spb_center_sample.pivot_table(index='publication_day', values='last_price').plot(grid=True);

In [ ]:
spb_center_sample.pivot_table(index='publication_month', values='last_price').plot(grid=True);

In [ ]:
spb_center_sample.pivot_table(index='publication_year', values='last_price').plot(grid=True);

Данные зависимости отличаются от общей выборки. Пик в среду и повышение цены при приближении к выходным. На уровне месяца так же пик цены осенью, а в срезе по годам есть тенденция к снижению цены с каждым годом

## Шаг 5. Общий вывод

Мы провели исследование объявлений по продаже недвижимости в Санкт-Петербурге и соседних населенных пунктах. Выяснили, что больше всего в продаже однокомнатных квартир площадью около 40 квадратных метров и высотой потолка 2.6 метра. В центре города преобладают более просторные квартиры, с высокими потолками, большей площадью, количеством комнат 2 и более и, соответственно стоят они дороже. Так же узнали, что чаще всего квартиры продаются менее чем за 100 дней, на цену влияют такие факторы как площадь, расстояние до центра города и сам населенный пункт. Наиболее благоприятное время для продажи квартиры - середина недели весной и осенью. Самый низкий спрос на квартиры летом.